# Runnables

No LangChain, os Runnables são componentes fundamentais que permitem a criação de cadeias de execução de tarefas, tornando o fluxo de dados mais eficiente e organizado. Entre os diversos tipos de Runnables disponíveis, três se destacam por sua funcionalidade e flexibilidade: RunnablePassthrough, RunnableLambda e RunnableParallel. Vamos explorar cada um deles.

## Métodos dos Runnables de LCEL

A interface padrão de LCEL inclui os seguintes métodos:

- `stream`: transmitir de volta fragmentos da resposta

- `invoke`: chamar a cadeia com um input

- `batch`: chamar a cadeia com uma lista de inputs

Esses também possuem métodos assíncronos correspondentes que devem ser usados com a sintaxe `asyncio await` para concorrência:

- `astream`: transmitir de volta fragmentos da resposta de forma assíncrona

- `ainvoke`: chamar a cadeia com um input de forma assíncrona

- `abatch`: chamar a cadeia com uma lista de inputs de forma assíncrona

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")

chain = prompt | model

## Invoke

O invoke é o método básico para inserir uma input na cadeia e receber uma resposta.

In [ ]:
chain.invoke({"assunto": "Inteligência Artificial"})

Ele pode ser rodado como uma simples string quando existe apenas uma input no prompt, mas a forma mais recomendada é informando especificamente o nome da input através de um dicionário.

In [ ]:
chain.invoke("cachorrinhos")

## Stream

Para recebermos uma saída conforme ela é gerada pelo modelo utilizamos o stream

In [ ]:
for stream in chain.stream("cachorrinhos"):
  print(stream.content, end="") 

## Batch

Para fazermos múltiplas requisições em paralelo utilizamos o batch

In [ ]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinhos'}, {'assunto': 'cavalinhos'}])

## Runnables especiais

### Rodando em paralelo
```
     Input      
      / \       
     /   \      
 Branch1 Branch2
     \   /      
      \ /       
      Combine   
```

### RunnableParallel

O RunnableParallel é um componente poderoso que permite executar múltiplos Runnables simultaneamente. Ele aceita um dicionário de Runnables e fornece a mesma entrada a todos eles, retornando um dicionário com os resultados correspondentes. Isso é especialmente útil quando você precisa realizar várias operações independentes ao mesmo tempo, economizando tempo em processos que podem ser feitos em paralelo.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Crie um nome para o seguinte produto: {produto}")

chain_nome = prompt | model | StrOutputParser()

In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Descreva o cliente potencial para o seguinte produto: {produto}")

chain_clientes = prompt | model | StrOutputParser()

In [ ]:
parallel = RunnableParallel({'nome_produto': chain_nome, 'publico': chain_clientes})
parallel.invoke({'produto': 'Um copo inquebrável'})

In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("""Dado o produto com o seguinte nome e seguinte
público potencial, desenvolva um anúncio para o produto.
                                          
Nome do produto: {nome_produto}
Público: {publico}""")

In [ ]:
chain = parallel | prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'produto': 'Um copo inquebrável'})

### RunnableLambda

O RunnableLambda permite que você crie um Runnable a partir de uma função Python arbitrária. Isso é útil para encapsular qualquer lógica que você queira aplicar aos dados ou para transformar entradas em saídas de uma maneira específica. Um aspecto interessante do RunnableLambda é que ele pode ser construído para funcionar de forma síncrona ou assíncrona, dependendo de como a função é definida.

In [ ]:
from langchain_core.runnables import RunnableLambda

def cumprimentar(nome):
    return f'Olá, {nome}!'

runnable_cumprimentar = RunnableLambda(cumprimentar)

resultado = runnable_cumprimentar.invoke('Maria')
print(resultado)

### RunnablePassthrough

O RunnablePassthrough é um tipo de Runnable que simplesmente passa a entrada recebida como saída, podendo também adicionar chaves adicionais ao resultado. Ele se comporta quase como uma função de identidade, mas é útil em cenários onde você deseja incluir lógica adicional ou manipular a entrada de alguma forma antes de passar adiante.

In [ ]:
from langchain_core.runnables import RunnablePassthrough
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("""Dado o produto com o seguinte nome e seguinte
público potencial, desenvolva um anúncio para o produto.
                                          
Nome do produto: {nome_produto}
Característica do produto: {produto}
Público: {publico}""")

parallel = RunnablePassthrough().assign(**{'nome_produto': chain_nome, 'publico': chain_clientes})
chain = parallel | prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'produto': 'Copo inquebrável'})